In [25]:
import json
import importlib
import random

from Project.utils.api import youtube_api as api
from Project.utils.storage import youtube_db as db
from Project.utils.analysis import text_classifier as tc

importlib.reload(api)
importlib.reload(db)
importlib.reload(tc)

<module 'Project.utils.analysis.text_classifier' from 'C:\\Users\\Filippo Corti\\Documents\\GitHub\\SocialMediaMining\\Project\\utils\\analysis\\text_classifier.py'>

In [26]:
credentials = json.load(open('../../keys/youtube_key.json'))

api_key = credentials['api_key']

trump_podcasts = [
    "xrFdHO7FH8w",
    "blqIZGXWUpU",
    "s11uIW7wi-E",
    "vC5cHjcgt5g",
    "G80iLTctFuY",
    "qCbfTN-caFI",
    "Ry1IjOft95c",
    "S7BTdUaNQM8",
    "1SsmPh8gCxU",
    "-dmwG54QsKc",
    "nwQil7tcImI",
    "G9lXnwuZ2qs",
    "hBMoPUAeLnY"
]

harris_podcasts = [
    "_KCRsjPCiCI",
    "bzThwqnQJDY",
    "7L4sts7I3xI",
    "pNbwMrBMGgE",
    "Vu5yD3fu6A8",
]

In [27]:
youtube = api.YoutubeApi(api_key)
youtube_db = db.SQLiteYoutubeSaver(db_name='../db/youtube.db')

In [4]:
videos = [
    youtube.get_video_by_id(video_id)
    for video_id in trump_podcasts + harris_podcasts
]

In [5]:
for video in videos:
    print(f"{video.channel_title} - {video.title}")

IMPAULSIVE - The Donald Trump Interview - IMPAULSIVE EP. 418
All-In Podcast - In conversation with President Trump
Adin Live - Adin Ross & Donald Trump FULL STREAM!
Theo Von - Donald Trump | This Past Weekend w/ Theo Von #526
Shawn Ryan Show - President Donald J. Trump - 47th President of the United States | SRS #127
Lex Fridman - Donald Trump Interview | Lex Fridman Podcast #442
FLAGRANT - Trump On Who Really Tried to Kill Him, Abortion & More
FULL SEND PODCAST - Donald Trump Calls Out Kamala Over FAKE Interview, Talks Joe Rogan, and Why He HATES Jimmy Kimmel!
Bussin' With The Boys - Donald Trump On How Success Will Unite America + The Future Of Media
PBD Podcast - Donald Trump Reveals His Next 5 Moves - Speaks On Tariffs, Obama & Iran | PBD Podcast | 489
OutKick - "Manhood Is Under ATTACK!" Why Donald Trump Keeps Fighting For America | Maintaining with Tyrus
Six Feet Under with Mark Calaway - Donald Trump Talks Pro Wrestling and What’s at Stake in 2024 | Six Feet Under #36
PowerfulJR

In [ ]:
for video in videos:
    youtube_db.insert_video(video)
    print(f"Extracting comments from {video.channel_title} - {video.title}")
    for comment in youtube.get_comments(video.id, threads_count=1000):
        youtube_db.insert_comment(comment)

In [28]:
text_classifier = tc.TextClassifier()

all_comments = youtube_db.get_all_comments()

In [29]:

all_comments = [
    comment
    for comment in all_comments
    if comment.video_id == trump_podcasts[12]
]

random.shuffle(all_comments)

In [30]:
print(len(all_comments))

14561


In [ ]:
current_batch = list() # Technically not a batch

for idx, comment in enumerate(all_comments):

    current_batch.append(comment)

    if len(current_batch) < 5 and idx != len(all_comments) - 1:
        continue

    # biases = text_classifier.get_political_bias(
    #     [comment.content for comment in current_batch]
    # )
    # leanings = text_classifier.get_political_leaning(
    #     [comment.content for comment in current_batch]
    # )
    # is_politicals = text_classifier.is_political(
    #     [comment.content for comment in current_batch]
    # )
    # sentiments = text_classifier.get_sentiment(
    #     [comment.content for comment in current_batch]
    # )
    # emotions = text_classifier.get_emotion(
    #     [comment.content for comment in current_batch]
    # )
    # llm_labels = text_classifier.get_llm_political_stance(
    #     candidates=["Trump" if comment.video_id in trump_podcasts else "Harris" for comment in current_batch],
    #     texts=[comment.content for comment in current_batch]
    # )
    gemini_labels = text_classifier.get_gemini_stance(
        candidates=["Trump" if comment.video_id in trump_podcasts else "Harris" for comment in current_batch],
        texts=[comment.content for comment in current_batch]
    )

    for i in range(len(current_batch)):
        comment_id = current_batch[i].id
        #bias = biases[i]
        #leaning = leanings[i]
        #is_political = is_politicals[i]
        #sentiment = sentiments[i]
        #emotion = emotions[i]
        #llm_label = llm_labels[i]
        gemini_label = gemini_labels[i]

        youtube_db.insert_comment_analysis(
            comment_id=comment_id,
            #bias=bias,
            #leaning=leaning,
            #is_political=is_political,
            #sentiment=sentiment,
            #emotion=emotion,
            #llm_label=llm_label,
            gemini_label=gemini_label,
        )

    current_batch = list()

    idx += 1
    if idx % 100 == 0:
        print(f"{idx}/{len(all_comments)}")

In [13]:
youtube_db.close()